In [1]:
import numpy as np
from time import time
from tqdm import tqdm

## Basic functions

In [2]:
def readfile(path):
    num_hor = 0
    counter = 0
    L = []
    L_reverse = []
    dic = {}
    f = open(path, "r")
    string = f.read().splitlines()
    f.close()
    num_pics = int(string[0])
    for i in range(num_pics):
        s = string[i+1].split(' ')
        l = [i,s[0],int(s[1])]
        if s[0] == 'H': num_hor+=1
        for tag in s[2:]:
            if tag in dic:
                tag_idx = dic[tag]
                l.append(tag_idx)
                L_reverse[tag_idx].append(i)
            else:
                l.append(counter)
                dic[tag] = counter
                L_reverse.append([i])
                counter+=1
        L.append(l)

    num_tags = counter

    return L, L_reverse, num_pics, num_tags, num_hor

In [3]:
def get_score(tags1, tags2):
    dic = {}
    for tags in [tags1,tags2]:
        for tag in tags:
            if tag in dic: dic[tag]+=1
            else: dic[tag] = 1

    num_dup = 0
    for key in dic:
        if dic[key] == 2: num_dup += 1
    num_1 = len(tags1) - num_dup
    num_2 = len(tags2) - num_dup

    return min(num_dup, num_1, num_2)

In [4]:
def get_intersec(tags1, tags2):
    dic = {}
    for tags in [tags1,tags2]:
        for tag in tags:
            if tag in dic: dic[tag]+=1
            else: dic[tag] = 1

    num_dup = 0
    for key in dic:
        if dic[key] == 2: num_dup += 1

    return num_dup

## Algorithms

In [5]:
def greedy_path_forest_simple(E, n, is_sorted=False):
    """
    V = [n]
    E is of the form [(u,v,w), ...], possible duplicates
    """
    
    score = 0
    degrees = [0]*n

    if not is_sorted: E.sort(key=lambda tup: tup[2], reverse=True)
    path_forest = []
    cycle_closing = [i for i in range(n)]
    min = 0
    max = len(E)

    for i in range(n-1):
        while True:
            if min>=max: break
            (u,v,w) = E[min]
            if (degrees[u] >= 2 or degrees[v] >=2 or cycle_closing[u] == v): min+=1
            else: break

        if min >= max: break
        score += w
        path_forest.append([u,v])
        cycle_closing[cycle_closing[u]], cycle_closing[cycle_closing[v]] = cycle_closing[v], cycle_closing[u]
        degrees[u]+=1
        degrees[v]+=1
        min+=1
    
    return path_forest, score

def greedy_path_forest(E, V, is_sorted=False):
    """ E is of the form [ (u,v,w), ...]"""
    n = len(V)
    V_reverse = {}
    for (i,v) in enumerate(V): V_reverse[v] = i
    E_simple = [(V_reverse[u], V_reverse[v], w) for (u,v,w) in E]
        
    path_forest_simple, score = greedy_path_forest_simple(E_simple, n, is_sorted)
    path_forest = []
    for [u1,u2] in path_forest_simple:
        v1, v2 = V[u1], V[u2]
        path_forest.append([v1,v2])
    return path_forest, score

In [6]:
def build_path_sequence_simple(path_forest, n):
    
    P = [[] for i in range(n)]
    for [u,v] in path_forest:
        P[u].append(v)
        P[v].append(u)

    deg = np.zeros(n)
    for u in range(n):
        deg[u] = len(P[u])

    degree_ones = []
    degree_zeros = []
    for u in range(n):
        if deg[u] == 1: degree_ones.append(u)
        if deg[u] == 0: degree_zeros.append(u)

    degree_ones_visited = {}
    for v in degree_ones:
        degree_ones_visited[v] = 0

    P_sequence = list(degree_zeros)
        
    for start in degree_ones:
        if degree_ones_visited[start] == 1: continue
        degree_ones_visited[start] = 1
        sec_last = start
        last = P[sec_last][0]
        P_sequence += [sec_last, last]

        while deg[last] > 1:
            [next1, next2] = P[last]
            if next1 == sec_last:
                P_sequence.append(next2)
                sec_last = last
                last = next2
            else:
                P_sequence.append(next1)
                sec_last = last
                last = next1

        degree_ones_visited[last] = 1

    return P_sequence

def build_path_sequence(path_forest, V):
    n = len(V)
    V_reverse = {}
    for (i,v) in enumerate(V): V_reverse[v] = i
    path_forest_simple = []
    for [u1,u2] in path_forest:
        v1, v2 = V_reverse[u1], V_reverse[u2]
        path_forest_simple.append([v1,v2])

    P_simple = build_path_sequence_simple(path_forest_simple, n)
    P = [V[u] for u in P_simple]
    return P

## Build Graphs

In [7]:
def build_graph_b(L, L_rev, num_pics):
    E = [set([]) for i in range(100)] #E[i] contains all edges of weight i
    
    for u in range(num_pics):
        tags1 = L[u][3:]
        possible_neighbours = set([])
        for tag in tags1:
            possible_neighbours.update(L_rev[tag])
        possible_neighbours = [x for x in possible_neighbours if x > u]

        for v in possible_neighbours:
            tags2 = L[v][3:]
            w = get_score(tags1,tags2)
            if w > 0: E[w].add((u,v,w))

    E = [e for F in reversed(E) for e in F]    
    return E

In [8]:
def build_graph_rand(n, L, deg, min_deg=1, N_pair=25):
    tags_list =[l[3:] for l in L]
    V_hor = []
    imgs_vert = []
    for i in range(n):
        l = L[i]
        if l[1] == 'H':
            V_hor.append(l[0])
        else:
            imgs_vert.append(l[0])

    def get_score_vx(x, y):
        if type(x) == int: tags1 = tags_list[x]
        else: tags1 = set(tags_list[x[0]]) | set(tags_list[x[1]])

        if type(y) == int: tags2 = tags_list[y]
        else: tags2 = set(tags_list[y[0]]) | set(tags_list[y[1]])

        return get_score(tags1, tags2)

    # build V_vert
    V_vert = []
    remaining_imgs = imgs_vert
    num_rem = len(remaining_imgs)
    added = np.zeros(n)
    update_threshold = num_rem / 2

    rand_size = 10**6
    random_imgs = np.random.choice(remaining_imgs, size=rand_size)
    rand_count = 0
    n_h = len(V_hor)
    for img1 in imgs_vert:
        if added[img1]: continue
        added[img1]=1
        candidates = []
        tries = 0
        while tries <= N_pair:
            if rand_count >= rand_size:
                rand_count = 0
                random_imgs = np.random.choice(remaining_imgs, size=rand_size)
            img2 = random_imgs[rand_count]
            rand_count += 1
            if added[img2] == 1: continue
            candidates.append((img2, get_intersec(tags_list[img1], tags_list[img2])))
            tries += 1

        (img2, w) = min(candidates, key= lambda tup: tup[1])
        added[img2] = 1
        if img1 > img2: [img1, img2] = [img2, img11]
        V_vert.append((img1, img2))
        num_rem = num_rem - 2

        if num_rem <= update_threshold:
            remaining_imgs = [j for j in remaining_imgs if added[j] == 0]
            update_threshold = num_rem / 2

        if num_rem == 0: break

    V = V_hor + V_vert
    E = [set([]) for i in range(100)] #E[i] contains all edges of weight i

    ## add random edges to E
    def leq(x, y):
        if type(x) == type(y): return x <= y
        else: return type(x) == int


    matching = [0] * n
    for (u1, u2) in V_vert:
        matching[u1] = u2
        matching[u2] = u1

    num_vxs = len(V)
    rand_count = 0
    random_idxs = np.random.randint(low=0, high=num_vxs, size=rand_size)
    
    for v1 in V:
        if rand_count + deg >= rand_size:
            rand_count = 0
            random_idxs = np.random.randint(low=0, high=num_vxs, size=rand_size)
        for idx in random_idxs[rand_count:rand_count + deg]:
            rand_count += 1
            v2 = V[idx]
            if v1 == v2: continue
            w = get_score_vx(v1,v2)
            if w >= min_deg:
                if leq(v1,v2): E[w].add((v1, v2, w))
                else: E[w].add((v2, v1, w))
                    
    E = [e for F in reversed(E) for e in F]   
    return V, E

## Submissions

In [9]:
path_a = "./qualification_round_2019.in/a_example.txt"
path_b = "./qualification_round_2019.in/b_lovely_landscapes.txt"
path_c = "./qualification_round_2019.in/c_memorable_moments.txt"
path_d = "./qualification_round_2019.in/d_pet_pictures.txt"
path_e = "./qualification_round_2019.in/e_shiny_selfies.txt"

In [10]:
def create_submission(P, path):
    n = len(P)
    s = str(n)
    for pic in P:
        if type(pic) == int:
            s += '\n'+ str(pic)
        else:
            s += '\n' + str(pic[0]) + ' ' + str(pic[1])
            
    f = open(path,"w+")
    f.write(s)
    f.close()
    return s

In [15]:
submit_path_b = './submissions/slideshow-b-temp.txt'
L, L_rev, num_pics, num_tags, num_hor = readfile(path_b)
E = build_graph_b(L, L_rev, num_pics)
print('Average Degree: ', 2*len(E)/num_pics)

path_forest, score = greedy_path_forest_simple(E, num_pics)
P = build_path_sequence_simple(path_forest, num_pics)
create_submission(P, submit_path_b)
print('Score: ', score)

Average Degree:  5.0
Score:  213639


In [16]:
submit_path_c = './submissions/slideshow-c-temp.txt'
L, L_rev, num_pics, num_tags, num_hor = readfile(path_c)

deg = 1000

V,E = build_graph_rand(num_pics, L, deg)
print('Average Degree: ', 2*len(E)/len(V))

path_forest, score = greedy_path_forest(E, V)
P = build_path_sequence(path_forest, V)
s = create_submission(P, submit_path_c)
print('Score: ', score)

Average Degree:  116.33866666666667
Score:  1503


In [13]:
submit_path_d = './submissions/slideshow-d-temp.txt'
L, L_rev, num_pics, num_tags, num_hor = readfile(path_d)

deg = 200 #Use 2000 and a good computer for better score

V,E = build_graph_rand(num_pics, L, deg)
print('Average Degree: ', 2*len(E)/len(V))

path_forest, score = greedy_path_forest(E, V, is_sorted=True)
P = build_path_sequence(path_forest, V)
s = create_submission(P, submit_path_d)
print('Score: ', score)

Average Degree:  355.142
Score:  383822


In [14]:
submit_path_e = './submissions/slideshow-e-temp.txt'
L, L_rev, num_pics, num_tags, num_hor = readfile(path_e)

deg = 200 #Use 3000 and a good computer for better score

V,E = build_graph_rand(num_pics, L, deg)
print('Average Degree: ', 2*len(E)/len(V))

path_forest, score = greedy_path_forest(E, V, is_sorted=True)
P = build_path_sequence(path_forest, V)
s = create_submission(P, submit_path_e)
print('Score: ', score)

Average Degree:  373.5645
Score:  303316
